# Segmenting nerve images

Here is the starting point for processing ultrasound nerve segmentation data from Kaggle

In [ ]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import glob
from skimage.io import imread
import matplotlib.pyplot as plt
IMAGE_DIR = '../../input/patient41/'

## Load and Preview Images

In [ ]:
us_image = plt.imread(IMAGE_DIR+'41_40.tif')
us_mask  = plt.imread(IMAGE_DIR+'41_40_mask.tif') != 0

In [ ]:
%matplotlib inline

fig, (ax1, ax2 ,ax3) = plt.subplots(1,3, figsize = (15,4))
ax1.imshow(us_image)
ax1.set_title('Ultrasound Image')
ax2.hist(us_image.ravel(),bins=50)
ax2.set_title('Ultrasound Intensity Distribution')
ax3.imshow(us_mask, cmap = 'bone')
ax3.set_title('Segmented Nerve');

## Just focus on the nerve
Show the segmentation of the nerve and the intensity inside that region

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2,2, figsize = (10,10))
ax1.imshow(us_image)
ax1.set_title('Ultrasound Image')
counts, bins, _ = ax2.hist(us_image.ravel(),bins=50)
ax2.set_title('Ultrasound Intensity Distribution')
ax3.imshow(us_image*us_mask)
ax3.set_title('Segmented Nerve Image')
ax4.hist(us_image[us_mask], bins = bins) # use the same bins again
ax4.set_title('Ultrasound Intensity Distribution')

# Simple Segmentation
We see too peaks in this histogram, lets try segmenting between them

In [ ]:
threshold = 120
thresholded = us_image > threshold
%matplotlib inline
fig, (ax1, ax2 ,ax3) = plt.subplots(1,3, figsize = (14,4))
ax1.imshow(us_image)
ax1.set_title('Ultrasound Image')
ax2.imshow(thresholded, interpolation='none')
ax2.set_title('Test Segmentation (>{0})'.format(threshold))
ax3.imshow(us_mask, cmap = 'bone')
ax3.set_title('Segmented Nerve')

# Scoring the Result
We can use the ROC curve to score the result and see how accurately we classified the images. Here the ROC curve is obtained using [SciKit Learn](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_curve.html).

In [ ]:
from sklearn.metrics import roc_curve, auc # roc curve tools

ground_truth_labels = us_mask.ravel() # we want to make them into vectors
score_value         = us_image.ravel()/255.0  # we want to make them into vectors and rescale to interval [0,1]

fpr, tpr, _ = roc_curve(ground_truth_labels,score_value)
roc_auc     = auc(fpr,tpr)

In [ ]:
# Visualization
fig, ax = plt.subplots(1,1)
ax.plot(fpr, tpr, label='ROC curve (area = {0:02f})'.format(roc_auc))
ax.plot([0, 1], [0, 1], 'k--')
ax.set_xlim([0.0, 1.0])
ax.set_ylim([0.0, 1.05])
ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')
ax.set_title('Receiver operating characteristic example')
ax.legend(loc="lower right");

# Gaussian Filter
Here we apply a gaussian filter and see how the performance changes

In [ ]:
%matplotlib inline
from skimage.filters import gaussian
gus_image = gaussian(us_image/255,sigma = 25)*255

test_threshold = 120
thresholded = gus_image>test_threshold
# Visualization
fig, (ax1, ax2 ,ax3) = plt.subplots(1,3, figsize = (14,4))
ax1.imshow(gus_image)
ax1.set_title('Gaussian Filtered Image')
ax2.imshow(thresholded, cmap = 'bone')
ax2.set_title('Test Segmentation')
ax3.imshow(us_mask, cmap = 'bone')
ax3.set_title('Segmented Nerve');

In [ ]:
ground_truth_labels = us_mask.ravel() # we want to make them into vectors
score_value = gus_image.ravel()/255.0 # we want to make them into vectors
fpr_gus, tpr_gus, _ = roc_curve(ground_truth_labels,score_value)
roc_auc_gus = auc(fpr_gus,tpr_gus)

# Visualization
fig, ax = plt.subplots(1,1)
ax.plot(fpr, tpr, label='Raw ROC curve (area = %0.2f)' % roc_auc)
ax.plot(fpr_gus, tpr_gus, label='Gaussian ROC curve (area = %0.2f)' % roc_auc_gus)
ax.plot([0, 1], [0, 1], 'k--')
ax.set_xlim([0.0, 1.0])
ax.set_ylim([0.0, 1.05])
ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')
ax.set_title('Receiver operating characteristic example')
ax.legend(loc="lower right");

# Tasks 

 1. How can you improve filtering in this analysis?
  - Which filter elements might improve the area under the ROC?
 3. Try making workflows to test out a few different filters
 - Where might morphological operations fit in?
 - How can you make them part of this workflow as well?
 6. (Challenge) Try and use the optimize toolbox of scipy with the fmin
    function (from scipy.optimize import fmin) to find the optimum
    parameters for the highers area (hint: fmin finds the minimum value)